In [ ]:
#Imports
import numpy as np
import sklearn
import pandas as pd
import random
import re
import scipy
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from scipy.stats import reciprocal, chi2_contingency
from functools import partial

In [ ]:
#Import data
data = pd.read_csv(".../codon_usage.csv",delimiter=",")

#Initialize Data
num_col_list = ['Ncodons','UUU','UUC','UUA','UUG','CUU','CUC','CUA','CUG','AUU','AUC','AUA','AUG','GUU','GUC','GUA','GUG','GCU','GCC','GCA','GCG','CCU','CCC','CCA','CCG','UGG','GGU','GGC','GGA','GGG','UCU','UCC','UCA','UCG','AGU','AGC','ACU','ACC','ACA','ACG','UAU','UAC','CAA','CAG','AAU','AAC','UGU','UGC','CAU','CAC','AAA','AAG','CGU','CGC','CGA','CGG','AGA','AGG','GAU','GAC','GAA','GAG','UAA','UAG','UGA']
#Data for reference frequencies
freqs = data[num_col_list]
kingdom = data['Kingdom']
freqs = np.array(freqs)
kingdom = np.array(kingdom)
#Data for phylogenetic classification
X = data[num_col_list]
y = data['Kingdom']
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#Exploring KMeans
def kmeans_eval(X):
    inertia_vals = [] #Generate lists
    silhouette = []
    for k in range(2,50): #Number of clusters 2 - 20
        model = KMeans(n_clusters = k, n_init = 10) #Initilize model, KMeans, 10 different initial centroids
        model.fit(X)
        s = silhouette_score(X,model.labels_)
        i = model.inertia_
        inertia_vals.append(i) #Append values
        silhouette.append(s)
        print(f'Number of clusters: {k}, Inertia: {np.around(i,3)}, Silhouette: {np.around(s,3)}')
        
    fig, ax = plt.subplots(2,1,figsize=(12,12))
    ax[0].set_title('Elbow Graph')  #Graph inertia like Figure 9-8
    ax[0].set_xlabel('Number of Clusters')
    ax[0].set_ylabel('Inertia / 10^11')
    ax[0].plot(range(2,50),inertia_vals,marker="*")
    ax[1].set_title('Silhouette for Different Clusters') #Graph silhouette like Figure 9-9
    ax[1].set_xlabel('Number of Clusters')
    ax[1].set_ylabel('Silhouette Score')
    ax[1].plot(range(2,50),silhouette,marker="*")
    plt.show


kmeans_eval(X_train) #Run 

#Look at prediction power for 'true' number of clusters
kmean_clf = KMeans(n_clusters = 5)
y_pred = kmean_clf.fit_predict(X_test)
#Manually calculate accuracy
err = y_pred - y_test
err = np.linalg.norm(err,0)
acc = err/(len(y_pred))
print((1 - acc) * 100) #output as %
#compare y_pred to y_test

In [ ]:
#Codon ranking
def linear_lasso(X,y,A,B,C): #builds lasso model to rank codon usage importance
    alphas = np.arange(A,B,C)
    N = len(alphas)
    coefs = np.zeros((N,65))
    for i in range(N):
        las = Lasso(alphas[i])
        las.fit(X,y)
        coefs[i,0] = alphas[i]
        coefs[i,1:] = las.coef_
    a = np.around(coefs,decimals=1)
    np.savetxt('lasso_importance.csv', a, delimiter=',') #exports excel file to rank
    return a
   
#Run function
linear_lasso(X,y,0.0000001,0.01,0.00001)
#Build and run best model
rf_best = RandomForestClassifier(
n_estimators = 533,
min_samples_split = 2,
min_samples_leaf = 1,
max_features = 'sqrt',
max_depth = None,
bootstrap = False)
rf_best.fit(X_train,y_train)
rf_best.score(X_test,y_test)

#Feature importance Random Forest
for name, score in zip(data[num_col_list], rf_best.feature_importances_):
    print(score)
#Exported to excel to rank

In [ ]:
#ORF detection
def random_RNA(length): #generate a random DNA sequence string of length = length
  global RNA
  RNA=""
  for count in range(length):
    RNA+=random.choice("CGUA")
  return RNA

class ORF_codon_frequencies: #Identifies all open reading frames in a RNA sequence and outputs each open reading frame codon usage frequency, evaulates with statistical test
  def __init__(self,seq,ref_freq): #Input RNA string, and reference frequency
    self.seq = seq
    self.ref_freq = ref_freq

  def potential_orfs(self): #Calculate all indexes of start and stop codons multiples of 3 away
    starts = [i.start() for i in re.finditer("AUG", self.seq)] #finds all indexes that have a start codon, etc.
    stopUAA = [i.start() for i in re.finditer("UAA", self.seq)]
    stopUAG = [i.start() for i in re.finditer("UAG", self.seq)]
    stopUGA = [i.start() for i in re.finditer("UGA", self.seq)]
    #pot_orf is the first index of the start codon and the first index of any corresponding stop codon
    global pot_orf
    pot_orf = []

    for i in range(len(starts)): #bottom 3 nested in first for loop, for each start codon try each stop
      for j in range(len(stopUAA)):
        if ((starts[i] - stopUAA[j]) % 3 == 0) and (starts[i] < stopUAA[j]): #requires multiple of 3 away and stop codon to be farther downstream to the start
          pot_orf.append(starts[i])
          pot_orf.append(stopUAA[j])

      for j in range(len(stopUAG)):
        if ((starts[i] - stopUAG[j]) % 3 == 0) and (starts[i] < stopUAG[j]):
          pot_orf.append(starts[i])
          pot_orf.append(stopUAG[j])

      for j in range(len(stopUGA)):
        if ((starts[i] - stopUGA[j]) % 3 == 0) and (starts[i] < stopUGA[j]):
          pot_orf.append(starts[i])
          pot_orf.append(stopUGA[j])
    
    global ORFs
    ORFs = []
    #ORFs string is the index of the A in AUG to the last letter of the stop codon
    for i in range(int(len(pot_orf) / 2)): 
      ORFs.append(self.seq[pot_orf[2*i]:(pot_orf[2*i+1]+3)]) #The plus 3 extends from the first letter of the stop to the last
    return ORFs

  def codon_frequencies(self,ORFs): #Input open reading frame string and outputs codon usage frequencies
    codon_list = ["UUU","UUC","UUA","UUG","CUU","CUC","CUA","CUG","AUU","AUC","AUA","AUG","GUU","GUC","GUA","GUG","GCU","GCC","GCA","GCG","CCU","CCC","CCA","CCG","UGG","GGU","GGC","GGA","GGG","UCU","UCC","UCA","UCG","AGU","AGC","ACU","ACC","ACA","ACG","UAU","UAC","CAA","CAG","AAU","AAC","UGU","UGC","CAU","CAC","AAA","AAG","CGU","CGC","CGA","CGG","AGA","AGG","GAU","GAC","GAA","GAG","UAA","UAG","UGA"]
    codon_total = int(len(ORFs) / 3)
    seq_codons = [ORFs[(3*i):(3*i+3)] for i in range(0, codon_total)]
    codon_freq = np.zeros((64,1)) #64 array rows for each frequency, corressponds to codon_list
    for i in range(len(codon_list)):
      codon_freq[i] = seq_codons.count(codon_list[i]) / codon_total
    return codon_freq

  def chisquare(self,ORFs,ORF_freqs,ref_freq):
    global chi_result
    chi_result = []
    #comb = []
    big_codon_list = [6,57,19,63,4,50,37,52,55,7] #list of most influential codons found from lasso and RF
    test_stat = 0
    p_val = 0
    '''
    for i in range(len(ORF_freqs)): #For testing all frequencies not just most influential
      test = np.copy(ORF_freqs[i] * len(ORFs[i]))
      ref = np.copy(self.ref_freq * len(ORFs[i]))
      for j in range(64):
        if ref_freq[j] == 0:
          test = np.delete(ORF_freqs[i],j)
          ref = np.delete(self.ref_freq,j)
      '''
    ref = ref_freq[big_codon_list,:]
    for i in range(len(ORF_freqs)):
      test = ORF_freqs[i][big_codon_list,:]
      test_stat, p_val = scipy.stats.chisquare(test,f_exp = ref) 
      if p_val < 0.05:
        chi_result.append(i)
        chi_result.append(test_stat)
        chi_result.append(p_val)

  def fit(self):
    ORFs = self.potential_orfs()
    global ORF_freqs
    ORF_freqs = {}
    for i in range(len(ORFs)):
      ORF_freqs[i] = self.codon_frequencies(ORFs[i])
    self.chisquare(ORFs,ORF_freqs,self.ref_freq)

In [ ]:
#Sample sequences
rep_rna = 'AAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGAAUGGUGA'
hsp_plasmid = 'GUCGUUCGGCUGCGGCGAGCGGUAUCAGCUCACUCAAAGGCGGUAAUACGGUUAUCCACAGAAUCAGGGGAUAACGCAGGAAAGAACAUGUGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGUAAAAAGGCCGCGUUGCUGGCGUUUUUCCAUAGGCUCCGCCCCCCUGACGAGCAUCACAAAAAUCGACGCUCAAGUCAGAGGUGGCGAAACCCGACAGGACUAUAAAGAUACCAGGCGUUUCCCCCUGGAAGCUCCCUCGUGCGCUCUCCUGUUCCGACCCUGCCGCUUACCGGAUACCUGUCCGCCUUUCUCCCUUCGGGAAGCGUGGCGCUUUCUCAUAGCUCACGCUGUAGGUAUCUCAGUUCGGUGUAGGUCGUUCGCUCCAAGCUGGGCUGUGUGCACGAACCCCCCGUUCAGCCCGACCGCUGCGCCUUAUCCGGUAACUAUCGUCUUGAGUCCAACCCGGUAAGACACGACUUAUCGCCACUGGCAGCAGCCACUGGUAACAGGAUUAGCAGAGCGAGGUAUGUAGGCGGUGCUACAGAGUUCUUGAAGUGGUGGCCUAACUACGGCUACACUAGAAGAACAGUAUUUGGUAUCUGCGCUCUGCUGAAGCCAGUUACCUUCGGAAAAAGAGUUGGUAGCUCUUGAUCCGGCAAACAAACCACCGCUGGUAGCGGUGGUUUUUUUGUUUGCAAGCAGCAGAUUACGCGCAGAAAAAAAGGAUCUCAAGAAGAUCCUUUGAUCUUUUCUACGGGGUCUGACGCUCAGUGGAACGAAAACUCACGUUAAGGGAUUUUGGUCAUGAGAUUAUCAAAAAGGAUCUUCACCUAGAUCCUUUUAAAUUAAAAAUGAAGUUUUAAAUCAAUCUAAAGUAUAUAUGAGUAAACUUGGUCUGACAGUUACCAAUGCUUAAUCAGUGAGGCACCUAUCUCAGCGAUCUGUCUAUUUCGUUCAUCCAUAGUUGCCUGACUCCCCGUCGUGUAGAUAACUACGAUACGGGAGGGCUUACCAUCUGGCCCCAGUGCUGCAAUGAUACCGCGAGACCCACGCUCACCGGCUCCAGAUUUAUCAGCAAUAAACCAGCCAGCCGGAAGGGCCGAGCGCAGAAGUGGUCCUGCAACUUUAUCCGCCUCCAUCCAGUCUAUUAAUUGUUGCCGGGAAGCUAGAGUAAGUAGUUCGCCAGUUAAUAGUUUGCGCAACGUUGUUGCCAUUGCUACAGGCAUCGUGGUGUCACGCUCGUCGUUUGGUAUGGCUUCAUUCAGCUCCGGUUCCCAACGAUCAAGGCGAGUUACAUGAUCCCCCAUGUUGUGCAAAAAAGCGGUUAGCUCCUUCGGUCCUCCGAUCGUUGUCAGAAGUAAGUUGGCCGCAGUGUUAUCACUCAUGGUUAUGGCAGCACUGCAUAAUUCUCUUACUGUCAUGCCAUCCGUAAGAUGCUUUUCUGUGACUGGUGAGUACUCAACCAAGUCAUUCUGAGAAUAGUGUAUGCGGCGACCGAGUUGCUCUUGCCCGGCGUCAAUACGGGAUAAUACCGCGCCACAUAGCAGAACUUUAAAAGUGCUCAUCAUUGGAAAACGUUCUUCGGGGCGAAAACUCUCAAGGAUCUUACCGCUGUUGAGAUCCAGUUCGAUGUAACCCACUCGUGCACCCAACUGAUCUUCAGCAUCUUUUACUUUCACCAGCGUUUCUGGGUGAGCAAAAACAGGAAGGCAAAAUGCCGCAAAAAAGGGAAUAAGGGCGACACGGAAAUGUUGAAUACUCAUACUCUUCCUUUUUCAAUAUUAUUGAAGCAUUUAUCAGGGUUAUUGUCUCAUGAGCGGAUACAUAUUUGAAUGUAUUUAGAAAAAUAAACAAAUAGGGGUUCCGCGCACAUUUCCCCGAAAAGUGCCACCUGACGUCGACGGAUCGGGAGAUCUCCCGAUCCCCUAUGGUGCACUCUCAGUACAAUCUGCUCUGAUGCCGCAUAGUUAAGCCAGUAUCUGCUCCCUGCUUGUGUGUUGGAGGUCGCUGAGUAGUGCGCGAGCAAAAUUUAAGCUACAACAAGGCAAGGCUUGACCGACAAUUGCAUGAAGAAUCUGCUUAGGGUUAGGCGUUUUGCGCUGCUUCGCGAUGUACGGGCCAGAUAUACGCGUUGACAUUGAUUAUUGACUAGUUAUUAAUAGUAAUCAAUUACGGGGUCAUUAGUUCAUAGCCCAUAUAUGGAGUUCCGCGUUACAUAACUUACGGUAAAUGGCCCGCCUGGCUGACCGCCCAACGACCCCCGCCCAUUGACGUCAAUAAUGACGUAUGUUCCCAUAGUAACGCCAAUAGGGACUUUCCAUUGACGUCAAUGGGUGGAGUAUUUACGGUAAACUGCCCACUUGGCAGUACAUCAAGUGUAUCAUAUGCCAAGUACGCCCCCUAUUGACGUCAAUGACGGUAAAUGGCCCGCCUGGCAUUAUGCCCAGUACAUGACCUUAUGGGACUUUCCUACUUGGCAGUACAUCUACGUAUUAGUCAUCGCUAUUACCAUGGUGAUGCGGUUUUGGCAGUACAUCAAUGGGCGUGGAUAGCGGUUUGACUCACGGGGAUUUCCAAGUCUCCACCCCAUUGACGUCAAUGGGAGUUUGUUUUGGCACCAAAAUCAACGGGACUUUCCAAAAUGUCGUAACAACUCCGCCCCAUUGACGCAAAUGGGCGGUAGGCGUGUACGGUGGGAGGUCUAUAUAAGCAGAGCUCUCUGGCUAACUAGAGAACCCACUGCUUACUGGCUUAUCGAAAUUAAUACGACUCACUAUAGGGAGACCCAAGCUUGGUACCGAGCUCGGAUCCACUAGUAACGGCCGCCAGUGUGCUGGAAUUCUGCAGAUAUCCAUCACACUGGCGGCCGCGCCAUGUACCCAUACGACGUCCCAGACUACGCUGAGGAAGUGCACCAUGGAGAGGAGGAGGUGGAGACUUUUGCCUUUCAGGCAGAAAUUGCCCAACUCAUGUCCCUCAUCAUCAAUACCUUCUAUUCCAACAAGGAGAUUUUCCUUCGGGAGUUGAUCUCUAAUGCUUCUGAUGCCUUGGACAAGAUUCGCUAUGAGAGCCUGACAGACCCUUCGAAGUUGGACAGUGGUAAAGAGCUGAAAAUUGACAUCAUCCCCAACCCUCAGGAACGUACCCUGACUUUGGUUGACACAGGCAUUGGCAUGACCAAAGCUGAUCUCAUAAAUAAUUUGGGAACCAUUGCCAAGUCUGGUACUAAAGCAUUCAUGGAGGCUCUUCAGGCUGGUGCAGACAUCUCCAUGAUUGGGCAGUUUGGUGUUGGCUUUUAUUCUGCCUACUUGGUGGCAGAGAAAGUGGUUGUGAUCACAAAGCACAACGAUGAUGAACAGUAUGCUUGGGAGUCUUCUGCUGGAGGUUCCUUCACUGUGCGUGCUGACCAUGGUGAGCCCAUUGGCAGGGGUACCAAAGUGAUCCUCCAUCUUAAAGAAGAUCAGACAGAGUACCUAGAAGAGAGGCGGGUCAAAGAAGUAGUGAAGAAGCAUUCUCAGUUCAUAGGCUAUCCCAUCACCCUUUAUUUGGAGAAGGAACGAGAGAAGGAAAUUAGUGAUGAUGAGGCAGAGGAAGAGAAAGGUGAGAAAGAAGAGGAAGAUAAAGAUGAUGAAGAAAAGCCCAAGAUCGAAGAUGUGGGUUCAGAUGAGGAGGAUGACAGCGGUAAGGAUAAGAAGAAGAAAACUAAGAAGAUCAAAGAGAAAUACAUUGAUCAGGAAGAACUAAACAAGACCAAGCCUAUUUGGACCAGAAACCCUGAUGACAUCACCCAAGAGGAGUAUGGAGAAUUCUACAAGAGCCUCACUAAUGACUGGGAAGACCACUUGGCAGUCAAGCACUUUUCUGUAGAAGGUCAGUUGGAAUUCAGGGCAUUGCUAUUUAUUCCUCGUCGGGCUCCCUUUGACCUUUUUGAGAACAAGAAGAAAAAGAACAACAUCAAACUCUAUGUCCGCCGUGUGUUCAUCAUGGACAGCUGUGAUGAGUUGAUACCAGAGUAUCUCAAUUUUAUCCGUGGUGUGGUUGACUCUGAGGAUCUGCCCCUGAACAUCUCCCGAGAAAUGCUCCAGCAGAGCAAAAUCUUGAAAGUCAUUCGCAAAAACAUUGUUAAGAAGUGCCUUGAGCUCUUCUCUGAGCUGGCAGAAGACAAGGAGAAUUACAAGAAAUUCUAUGAGGCAUUCUCUAAAAAUCUCAAGCUUGGAAUCCACGAAGACUCCACUAACCGCCGCCGCCUGUCUGAGCUGCUGCGCUAUCAUACCUCCCAGUCUGGAGAUGAGAUGACAUCUCUGUCAGAGUAUGUUUCUCGCAUGAAGGAGACACAGAAGUCCAUCUAUUACAUCACUGGUGAGAGCAAAGAGCAGGUGGCCAACUCAGCUUUUGUGGAGCGAGUGCGGAAACGGGGCUUCGAGGUGGUAUAUAUGACCGAGCCCAUUGACGAGUACUGUGUGCAGCAGCUCAAGGAAUUUGAUGGGAAGAGCCUGGUCUCAGUUACCAAGGAGGGUCUGGAGCUGCCUGAGGAUGAGGAGGAGAAGAAGAAGAUGGAAGAGAGCAAGGCAAAGUUUGAGAACCUCUGCAAGCUCAUGAAAGAAAUCUUAGAUAAGAAGGUUGAGAAGGUGACAAUCUCCAAUAGACUUGUGUCUUCACCUUGCUGCAUUGUGACCAGCACCUACGGCUGGACAGCCAAUAUGGAGCGGAUCAUGAAAGCCCAGGCACUUCGGGACAACUCCACCAUGGGCUAUAUGAUGGCCAAAAAGCACCUGGAGAUCAACCCUGACCACCCCAUUGUGGAGACGCUGCGGCAGAAGGCUGAGGCCGACAAGAAUGAUAAGGCAGUUAAGGACCUGGUGGUGCUGCUGUUUGAAACCGCCCUGCUAUCUUCUGGCUUUUCCCUUGAGGAUCCCCAGACCCACUCCAACCGCAUCUAUCGCAUGAUCAAGCUAGGUCUAGGUAUUGAUGAAGAUGAAGUGGCAGCAGAGGAACCCAAUGCUGCAGUUCCUGAUGAGAUCCCCCCUCUCGAGGGCGAUGAGGAUGCGUCUCGCAUGGAAGAAGUCGAUUAGUCUAGAGGGCCCUAUUCUAUAGUGUCACCUAAAUGCUAGAGCUCGCUGAUCAGCCUCGACUGUGCCUUCUAGUUGCCAGCCAUCUGUUGUUUGCCCCUCCCCCGUGCCUUCCUUGACCCUGGAAGGUGCCACUCCCACUGUCCUUUCCUAAUAAAAUGAGGAAAUUGCAUCGCAUUGUCUGAGUAGGUGUCAUUCUAUUCUGGGGGGUGGGGUGGGGCAGGACAGCAAGGGGGAGGAUUGGGAAGACAAUAGCAGGCAUGCUGGGGAUGCGGUGGGCUCUAUGGCUUCUGAGGCGGAAAGAACCAGCUGGGGCUCUAGGGGGUAUCCCCACGCGCCCUGUAGCGGCGCAUUAAGCGCGGCGGGUGUGGUGGUUACGCGCAGCGUGACCGCUACACUUGCCAGCGCCCUAGCGCCCGCUCCUUUCGCUUUCUUCCCUUCCUUUCUCGCCACGUUCGCCGGCUUUCCCCGUCAAGCUCUAAAUCGGGGGCUCCCUUUAGGGUUCCGAUUUAGUGCUUUACGGCACCUCGACCCCAAAAAACUUGAUUAGGGUGAUGGUUCACGUAGUGGGCCAUCGCCCUGAUAGACGGUUUUUCGCCCUUUGACGUUGGAGUCCACGUUCUUUAAUAGUGGACUCUUGUUCCAAACUGGAACAACACUCAACCCUAUCUCGGUCUAUUCUUUUGAUUUAUAAGGGAUUUUGCCGAUUUCGGCCUAUUGGUUAAAAAAUGAGCUGAUUUAACAAAAAUUUAACGCGAAUUAAUUCUGUGGAAUGUGUGUCAGUUAGGGUGUGGAAAGUCCCCAGGCUCCCCAGCAGGCAGAAGUAUGCAAAGCAUGCAUCUCAAUUAGUCAGCAACCAGGUGUGGAAAGUCCCCAGGCUCCCCAGCAGGCAGAAGUAUGCAAAGCAUGCAUCUCAAUUAGUCAGCAACCAUAGUCCCGCCCCUAACUCCGCCCAUCCCGCCCCUAACUCCGCCCAGUUCCGCCCAUUCUCCGCCCCAUGGCUGACUAAUUUUUUUUAUUUAUGCAGAGGCCGAGGCCGCCUCUGCCUCUGAGCUAUUCCAGAAGUAGUGAGGAGGCUUUUUUGGAGGCCUAGGCUUUUGCAAAAAGCUCCCGGGAGCUUGUAUAUCCAUUUUCGGAUCUGAUCAAGAGACAGGAUGAGGAUCGUUUCGCAUGAUUGAACAAGAUGGAUUGCACGCAGGUUCUCCGGCCGCUUGGGUGGAGAGGCUAUUCGGCUAUGACUGGGCACAACAGACAAUCGGCUGCUCUGAUGCCGCCGUGUUCCGGCUGUCAGCGCAGGGGCGCCCGGUUCUUUUUGUCAAGACCGACCUGUCCGGUGCCCUGAAUGAACUGCAGGACGAGGCAGCGCGGCUAUCGUGGCUGGCCACGACGGGCGUUCCUUGCGCAGCUGUGCUCGACGUUGUCACUGAAGCGGGAAGGGACUGGCUGCUAUUGGGCGAAGUGCCGGGGCAGGAUCUCCUGUCAUCUCACCUUGCUCCUGCCGAGAAAGUAUCCAUCAUGGCUGAUGCAAUGCGGCGGCUGCAUACGCUUGAUCCGGCUACCUGCCCAUUCGACCACCAAGCGAAACAUCGCAUCGAGCGAGCACGUACUCGGAUGGAAGCCGGUCUUGUCGAUCAGGAUGAUCUGGACGAAGAGCAUCAGGGGCUCGCGCCAGCCGAACUGUUCGCCAGGCUCAAGGCGCGCAUGCCCGACGGCGAGGAUCUCGUCGUGACCCAUGGCGAUGCCUGCUUGCCGAAUAUCAUGGUGGAAAAUGGCCGCUUUUCUGGAUUCAUCGACUGUGGCCGGCUGGGUGUGGCGGACCGCUAUCAGGACAUAGCGUUGGCUACCCGUGAUAUUGCUGAAGAGCUUGGCGGCGAAUGGGCUGACCGCUUCCUCGUGCUUUACGGUAUCGCCGCUCCCGAUUCGCAGCGCAUCGCCUUCUAUCGCCUUCUUGACGAGUUCUUCUGAGCGGGACUCUGGGGUUCGAAAUGACCGACCAAGCGACGCCCAACCUGCCAUCACGAGAUUUCGAUUCCACCGCCGCCUUCUAUGAAAGGUUGGGCUUCGGAAUCGUUUUCCGGGACGCCGGCUGGAUGAUCCUCCAGCGCGGGGAUCUCAUGCUGGAGUUCUUCGCCCACCCCAACUUGUUUAUUGCAGCUUAUAAUGGUUACAAAUAAAGCAAUAGCAUCACAAAUUUCACAAAUAAAGCAUUUUUUUCACUGCAUUCUAGUUGUGGUUUGUCCAAACUCAUCAAUGUAUCUUAUCAUGUCUGUAUACCGUCGACCUCUAGCUAGAGCUUGGCGUAAUCAUGGUCAUAGCUGUUUCCUGUGUGAAAUUGUUAUCCGCUCACAAUUCCACACAACAUACGAGCCGGAAGCAUAAAGUGUAAAGCCUGGGGUGCCUAAUGAGUGAGCUAACUCACAUUAAUUGCGUUGCGCUCACUGCCCGCUUUCCAGUCGGGAAACCUGUCGUGCCAGCUGCAUUAAUGAAUCGGCCAACGCGCGGGGAGAGGCGGUUUGCGUAUUGGGCGCUCUUCCGCUUCCUCGCUCACUGACUCGCUGCGCUCG'
puc18_rna = 'ACGAAAGGGCCUCGUGAUACGCCUAUUUUUAUAGGUUAAUGUCAUGAUAAUAAUGGUUUCUUAGACGUCAGGUGGCACUUUUCGGGGAAAUGUGCGCGGAACCCCUAUUUGUUUAUUUUUCUAAAUACAUUCAAAUAUGUAUCCGCUCAUGAGACAAUAACCCUGAUAAAUGCUUCAAUAAUAUUGAAAAAGGAAGAGUAUGAGUAUUCAACAUUUCCGUGUCGCCCUUAUUCCCUUUUUUGCGGCAUUUUGCCUUCCUGUUUUUGCUCACCCAGAAACGCUGGUGAAAGUAAAAGAUGCUGAAGAUCAGUUGGGUGCACGAGUGGGUUACAUCGAACUGGAUCUCAACAGCGGUAAGAUCCUUGAGAGUUUUCGCCCCGAAGAACGUUUUCCAAUGAUGAGCACUUUUAAAGUUCUGCUAUGUGGCGCGGUAUUAUCCCGUAUUGACGCCGGGCAAGAGCAACUCGGUCGCCGCAUACACUAUUCUCAGAAUGACUUGGUUGAGUACUCACCAGUCACAGAAAAGCAUCUUACGGAUGGCAUGACAGUAAGAGAAUUAUGCAGUGCUGCCAUAACCAUGAGUGAUAACACUGCGGCCAACUUACUUCUGACAACGAUCGGAGGACCGAAGGAGCUAACCGCUUUUUUGCACAACAUGGGGGAUCAUGUAACUCGCCUUGAUCGUUGGGAACCGGAGCUGAAUGAAGCCAUACCAAACGACGAGCGUGACACCACGAUGCCUGUAGCAAUGGCAACAACGUUGCGCAAACUAUUAACUGGCGAACUACUUACUCUAGCUUCCCGGCAACAAUUAAUAGACUGGAUGGAGGCGGAUAAAGUUGCAGGACCACUUCUGCGCUCGGCCCUUCCGGCUGGCUGGUUUAUUGCUGAUAAAUCUGGAGCCGGUGAGCGUGGGUCUCGCGGUAUCAUUGCAGCACUGGGGCCAGAUGGUAAGCCCUCCCGUAUCGUAGUUAUCUACACGACGGGGAGUCAGGCAACUAUGGAUGAACGAAAUAGACAGAUCGCUGAGAUAGGUGCCUCACUGAUUAAGCAUUGGUAACUGUCAGACCAAGUUUACUCAUAUAUACUUUAGAUUGAUUUAAAACUUCAUUUUUAAUUUAAAAGGAUCUAGGUGAAGAUCCUUUUUGAUAAUCUCAUGACCAAAAUCCCUUAACGUGAGUUUUCGUUCCACUGAGCGUCAGACCCCGUAGAAAAGAUCAAAGGAUCUUCUUGAGAUCCUUUUUUUCUGCGCGUAAUCUGCUGCUUGCAAACAAAAAAACCACCGCUACCAGCGGUGGUUUGUUUGCCGGAUCAAGAGCUACCAACUCUUUUUCCGAAGGUAACUGGCUUCAGCAGAGCGCAGAUACCAAAUACUGUUCUUCUAGUGUAGCCGUAGUUAGGCCACCACUUCAAGAACUCUGUAGCACCGCCUACAUACCUCGCUCUGCUAAUCCUGUUACCAGUGGCUGCUGCCAGUGGCGAUAAGUCGUGUCUUACCGGGUUGGACUCAAGACGAUAGUUACCGGAUAAGGCGCAGCGGUCGGGCUGAACGGGGGGUUCGUGCACACAGCCCAGCUUGGAGCGAACGACCUACACCGAACUGAGAUACCUACAGCGUGAGCUUUGAGAAAGCGCCACGCUUCCCGAAGGGAGAAAGGCGGACAGGUAUCCGGUAAGCGGCAGGGUCGGAACAGGAGAGCGCACGAGGGAGCUUCCAGGGGGAAACGCCUGGUAUCUUUAUAGUCCUGUCGGGUUUCGCCACCUCUGACUUGAGCGUCGAUUUUUGUGAUGCUCGUCAGGGGGGCGGAGCCUAUGGAAAAACGCCAGCAACGCGGCCUUUUUACGGUUCCUGGCCUUUUGCUGGCCUUUUGCUCACAUGUUCUUUCCUGCGUUAUCCCCUGAUUCUGUGGAUAACCGUAUUACCGCCUUUGAGUGAGCUGAUACCGCUCGCCGCAGCCGAACGACCGAGCGCAGCGAGUCAGUGAGCGAGGAAGCGGAAGAGCGCCCAAUACGCAAACCGCCUCUCCCCGCGCGUUGGCCGAUUCAUUAAUGCAGCUGGCACGACAGGUUUCCCGACUGGAAAGCGGGCAGUGAGCGCAACGCAAUUAAUGUGAGUUAGCUCACUCAUUAGGCACCCCAGGCUUUACACUUUAUGCUUCCGGCUCGUAUGUUGUGUGGAAUUGUGAGCGGAUAACAAUUUCACACAGGAAACAGCUAUGACCAUGAUUACGAAUUCGAGCUCGGUACCCGGGGAUCCUCUAGAGUCGACCUGCAGGCAUGCAAGCUUGGCACUGGCCGUCGUUUUACAACGUCGUGACUGGGAAAACCCUGGCGUUACCCAACUUAAUCGCCUUGCAGCACAUCCCCCUUUCGCCAGCUGGCGUAAUAGCGAAGAGGCCCGCACCGAUCGCCCUUCCCAACAGUUGCGCAGCCUGAAUGGCGAAUGGCGCCUGAUGCGGUAUUUUCUCCUUACGCAUCUGUGCGGUAUUUCACACCGCAUAUGGUGCACUCUCAGUACAAUCUGCUCUGAUGCCGCAUAGUUAAGCCAGCCCCGACACCCGCCAACACCCGCUGACGCGCCCUGACGGGCUUGUCUGCUCCCGGCAUCCGCUUACAGACAAGCUGUGACCGUCUCCGGGAGCUGCAUGUGUCAGAGGUUUUCACCGUCAUCACCGAAACGCG'

#Calcualte ORFs
p1 = ORF_codon_frequencies(puc18_rna,e_coli_freq)
p1.fit()
len(ORFs)

#Print results
print(len(chi_result) / 3) #Total 
print(chi_result)

#For sorting ORF lengths
orf_len = []
for i in range(len(ORFs)):
  orf_len.append(len(ORFs[i]))
orf_len.sort()
print(orf_len)

#Identify possible potential ORFs are are correct
for i in range(len(ORFs)):
  if len(ORFs[i]) == 324:
    print(f'Possible Bgal {i}')
  if len(ORFs[i]) == 861:
    print(f'Possible AMPr {i}')
  if len(ORFs[i]) == 2199:
    print(f'Possible HSP90 {i}')